Import libraries.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import rvb

Define user inputs. These will come from the questionnaire.

In [ ]:
num_years = 20
down_payment = 200000.0
initial_monthly_budget = 3200.0
annual_raise = 0.03

Compute montly budget over time.

In [ ]:
year = np.arange(1, num_years + 1)
monthly_budget = initial_monthly_budget * (1 + annual_raise) ** (year - 1)

Get home/rent prices. Here, I randomize for demonstration purposes. The home prices range from \$550k to \$650k. The rent prices range from 0.45% to 0.55% of the home price. In practice, of course, we will get these values from the datasets.

In [ ]:
np.random.seed(0)
num_homes = 100
home_price = np.random.uniform(550000.0, 650000.0, size=(num_homes, 1))
rent_multiplier = np.random.uniform(0.0045, 0.0055, size=home_price.shape)
rent_price = home_price * rent_multiplier

Create an instance of the Renter object. Set the rent price.

In [ ]:
renter = rvb.Renter()
renter.rent_price = rent_price

Create an instance of the Buyer object. Set the home price and down payment.

In [ ]:
buyer = rvb.Buyer()
buyer.home_price = home_price
buyer.down_payment = down_payment

Calculate net value of the renter and the buyer over time.

In [ ]:
renter_net_value = rvb.calculate_renter_net_value(renter, buyer, num_years)
buyer_net_value = rvb.calculate_buyer_net_value(buyer, num_years)

Calculate average monthly costs over time.

In [ ]:
renter_monthly_costs = renter.calculate_annual_costs(year) / 12
buyer_monthly_costs = buyer.calculate_annual_costs(year) / 12

Just a couple plotting helper routines.

In [ ]:
def plot_minmax(ax, x, y, label=None):
    line, = ax.plot(x, y.mean(axis=0), label=label)
    color = line.get_color()
    ax.fill_between(x, y.min(axis=0), y.max(axis=0), alpha=0.3,
                    facecolor=color, edgecolor='none')
    return ax   

def format_price(axis):
    axis.set_major_formatter(plt.FuncFormatter(
        rvb.visualize.price_formatter))

Plot net value over time of renting versus buying. The solid line represents the mean of all homes. The extent of the filled region represents the min and max.

In [ ]:
with rvb.visualize.style():
    fig = plt.figure(1)
    fig.clf()
    ax = fig.gca()
    plot_minmax(ax, year, renter_net_value, label='Rent')
    plot_minmax(ax, year, buyer_net_value, label='Buy')
    ymin, ymax = ax.get_ylim()
    ymax = ymax + 0.3 * (ymax - ymin)
    ax.set_xlim(1, num_years)
    ax.set_ylim(ymin, ymax)
    ax.legend(loc='upper right', frameon=True, framealpha=1.0,
              facecolor='w')
    format_price(ax.yaxis)
    ax.set_xlabel('Year')
    ax.set_ylabel('Net Value');

Plot monthly costs over time of renting versus buying. Note that the budget line intersects the min-max monthly costs curves, meaning some of these homes are too expensive to fit in the user's budget.

In [ ]:
with rvb.visualize.style():
    fig = plt.figure(2)
    fig.clf()
    ax = fig.gca()
    plot_minmax(ax, year, renter_monthly_costs, label='Rent')
    plot_minmax(ax, year, buyer_monthly_costs, label='Buy')
    ax.plot(year, monthly_budget, color='k', linestyle='--',
            label='Budget')
    ymin, ymax = ax.get_ylim()
    ymax = ymax + 0.3 * (ymax - ymin)
    ax.set_xlim(1, num_years)
    ax.set_ylim(ymin, ymax)
    ax.legend(loc='upper right', frameon=True, framealpha=1.0,
              facecolor='w')
    format_price(ax.yaxis)
    ax.set_xlabel('Year')
    ax.set_ylabel('Monthly Costs');

Determine for which homes the monthly costs of renting or buying exceed the monthly budget.

In [ ]:
renter_over_budget = (renter_monthly_costs > monthly_budget).any(axis=1)
buyer_over_budget = (buyer_monthly_costs > monthly_budget).any(axis=1)

Define the score of renting/buying as the net value in the final year. Where over budget, set score to -$\infty$ so it is effectively masked out in the comparison.

In [ ]:
renter_score = np.where(renter_over_budget, -np.inf, 
                        renter_net_value[:, -1])
buyer_score = np.where(buyer_over_budget, -np.inf, buyer_net_value[:, -1])

Define the decision criteria.  If the net value of buying is greater or equal to that of renting, we recommend the user BUY. Otherwise, the user should RENT. If the monthly costs of renting and buying both exceed the monthly budget for any year, than we recommend the user neither rent nor buy (EXCLUDE).

In [ ]:
recommend_buy = (buyer_score >= renter_score) & ~buyer_over_budget
recommend_rent = (buyer_score < renter_score) & ~renter_over_budget
exclude = renter_over_budget & buyer_over_budget

labels = np.empty(num_homes, dtype=np.int)
labels[recommend_buy] = 1
labels[recommend_rent] = 2
labels[exclude] = 3

Here we just visualize the distribution of labels. Note that despite the score of renting being lower for all homes than the score of buying, we recommend the user RENT where the montly costs of buying are too high. Without this constraint, we would always make the recommendation to BUY.

In [ ]:
with rvb.visualize.style():
    fig = plt.figure(3)
    fig.clf()
    ax = fig.gca()
    for label_int, label_str in zip([2, 1, 3], ['Rent', 'Buy', 'Exclude']):
        cond = labels == label_int
        ax.scatter(home_price[cond], rent_price[cond], label=label_str)
    ymin, ymax = ax.get_ylim()
    ymax = ymax + 0.3 * (ymax - ymin)
    ax.set_ylim(ymin, ymax)
    ax.legend(loc='upper right', frameon=True, framealpha=1.0,
              facecolor='w')
    format_price(ax.xaxis)
    format_price(ax.yaxis)
    ax.set_xlabel('Home Price')
    ax.set_ylabel('Rent Price');